In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
 
data = pd.read_csv("parkinsons.csv")
   
print (data.head(10))

             name  MDVP:Fo(Hz)  MDVP:Fhi(Hz)  MDVP:Flo(Hz)  MDVP:Jitter(%)  \
0  phon_R01_S01_1      119.992       157.302        74.997         0.00784   
1  phon_R01_S01_2      122.400       148.650       113.819         0.00968   
2  phon_R01_S01_3      116.682       131.111       111.555         0.01050   
3  phon_R01_S01_4      116.676       137.871       111.366         0.00997   
4  phon_R01_S01_5      116.014       141.781       110.655         0.01284   
5  phon_R01_S01_6      120.552       131.162       113.787         0.00968   
6  phon_R01_S02_1      120.267       137.244       114.820         0.00333   
7  phon_R01_S02_2      107.332       113.840       104.315         0.00290   
8  phon_R01_S02_3       95.730       132.068        91.754         0.00551   
9  phon_R01_S02_4       95.056       120.103        91.226         0.00532   

   MDVP:Jitter(Abs)  MDVP:RAP  MDVP:PPQ  Jitter:DDP  MDVP:Shimmer  ...  \
0           0.00007   0.00370   0.00554     0.01109       0.04374  

In [1]:
<!-- action="{{ url_for('predict')}}"method="post" -->

SyntaxError: invalid syntax (3296619564.py, line 1)

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# from google.colab import drive
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import (
    classification_report, precision_score, recall_score, f1_score,
    roc_auc_score, accuracy_score, confusion_matrix
)
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
import pickle

In [7]:
df = pd.read_csv("parkinsons.csv")

def precision(class_id, TP, FP, TN, FN):
    sonuc = 0
    for i in range(len(class_id)):
        if TP[i] == 0 or FP[i] == 0:
            TP[i] = 0.00000001
            FP[i] = 0.00000001
        sonuc += (TP[i] / (TP[i] + FP[i]))

    sonuc = sonuc / len(class_id)
    return sonuc


In [8]:
def recall(class_id, TP, FP, TN, FN):
    sonuc = 0
    for i in range(len(class_id)):
        if TP[i] == 0 or FN[i] == 0:
            TP[i] = 0.00000001
            FN[i] = 0.00000001
        sonuc += (TP[i] / (TP[i] + FN[i]))

    sonuc = sonuc / len(class_id)
    return sonuc

In [9]:
def accuracy(class_id, TP, FP, TN, FN):
    sonuc = 0
    for i in range(len(class_id)):
        sonuc += ((TP[i] + TN[i]) / (TP[i] + FP[i] + TN[i] + FN[i]))

    sonuc = sonuc / len(class_id)
    return sonuc


In [10]:
def specificity(class_id, TP, FP, TN, FN):
    sonuc = 0
    for i in range(len(class_id)):
        if TN[i] == 0 or FP[i] == 0:
            TN[i] = 0.00000001
            FP[i] = 0.00000001
        sonuc += (TN[i] / (FP[i] + TN[i]))

    sonuc = sonuc / len(class_id)
    return sonuc

In [11]:
def NPV(class_id, TP, FP, TN, FN):
    sonuc = 0
    for i in range(len(class_id)):
        if TN[i] == 0 or FN[i] == 0:
            TN[i] = 0.00000001
            FN[i] = 0.00000001
        sonuc += (TN[i] / (TN[i] + FN[i]))

    sonuc = sonuc / len(class_id)
    return sonuc

In [12]:
def perf_measure(y_actual, y_pred):
    class_id = set(y_actual).union(set(y_pred))
    TP = []
    FP = []
    TN = []
    FN = []
    for index, _id in enumerate(class_id):
        TP.append(0)
        FP.append(0)
        TN.append(0)
        FN.append(0)
    for i in range(len(y_pred)):
        if y_actual[i] == y_pred[i] == _id:
            TP[index] += 1
        if y_pred[i] == _id and y_actual[i] != y_pred[i]:
            FP[index] += 1
        if y_pred[i] != _id and y_actual[i] == _id:
            FN[index] += 1
        if y_pred[i] != _id and y_actual[i] != _id:
            TN[index] += 1
    precision_score_val = precision(class_id, TP, FP, TN, FN)
    recall_score_val = recall(class_id, TP, FP, TN, FN)
    accuracy_score_val = accuracy(class_id, TP, FP, TN, FN)
    specificity_score_val = specificity(class_id, TP, FP, TN, FN)
    npv_score_val = NPV(class_id, TP, FP, TN, FN)

    return (
        precision_score_val, recall_score_val, accuracy_score_val,
        specificity_score_val, npv_score_val
    )



In [33]:
X = df.drop(columns=['name']).values
y = df['name'].values

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195 entries, 0 to 194
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              195 non-null    object 
 1   MDVP:Fo(Hz)       195 non-null    float64
 2   MDVP:Fhi(Hz)      195 non-null    float64
 3   MDVP:Flo(Hz)      195 non-null    float64
 4   MDVP:Jitter(%)    195 non-null    float64
 5   MDVP:Jitter(Abs)  195 non-null    float64
 6   MDVP:RAP          195 non-null    float64
 7   MDVP:PPQ          195 non-null    float64
 8   Jitter:DDP        195 non-null    float64
 9   MDVP:Shimmer      195 non-null    float64
 10  MDVP:Shimmer(dB)  195 non-null    float64
 11  Shimmer:APQ3      195 non-null    float64
 12  Shimmer:APQ5      195 non-null    float64
 13  MDVP:APQ          195 non-null    float64
 14  Shimmer:DDA       195 non-null    float64
 15  NHR               195 non-null    float64
 16  HNR               195 non-null    float64
 1

In [36]:
selector = SelectKBest(score_func=f_classif, k=10)
X = selector.fit_transform(X, y)

c:\Users\Sai Manne\anaconda3\envs\myenv\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:109: RuntimeWarning: invalid value encountered in divide
  msw = sswn / float(dfwn)


In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [20]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
knn_y_pred = knn.predict(X_test)


In [21]:
gnb = GaussianNB()
gnb.fit(X_train, y_train)
gnb_y_pred = gnb.predict(X_test)

In [22]:
lr = LogisticRegression(random_state=0)
lr.fit(X_train, y_train)
lr_y_pred = lr.predict(X_test)

c:\Users\Sai Manne\anaconda3\envs\myenv\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [23]:
knn_scores = perf_measure(y_test, knn_y_pred)
gnb_scores = perf_measure(y_test, gnb_y_pred)
lr_scores = perf_measure(y_test, lr_y_pred)


In [24]:
print("K-Nearest Neighbors (KNN):")
print(f"Precision: {knn_scores[0]}")
print(f"Recall: {knn_scores[1]}")
print(f"Accuracy: {knn_scores[2]}")
print(f"Specificity: {knn_scores[3]}")
print(f"NPV: {knn_scores[4]}")


K-Nearest Neighbors (KNN):
Precision: 0.5
Recall: 0.491803278852459
Accuracy: 0.3438419503953395
Specificity: 0.5081967213071613
NPV: 0.5077763766288357


In [25]:
print("\nNaive Bayes:")
print(f"Precision: {gnb_scores[0]}")
print(f"Recall: {gnb_scores[1]}")
print(f"Accuracy: {gnb_scores[2]}")
print(f"Specificity: {gnb_scores[3]}")
print(f"NPV: {gnb_scores[4]}")


Naive Bayes:
Precision: 0.5
Recall: 0.4931506850684932
Accuracy: 0.34211450649473557
Specificity: 0.5068493150648883
NPV: 0.5064980681419037


In [26]:
print("\nLogistic Regression:")
print(f"Precision: {lr_scores[0]}")
print(f"Recall: {lr_scores[1]}")
print(f"Accuracy: {lr_scores[2]}")
print(f"Specificity: {lr_scores[3]}")
print(f"NPV: {lr_scores[4]}")


Logistic Regression:
Precision: 0.5
Recall: 0.49242424257575756
Accuracy: 0.34304584304215713
Specificity: 0.5075757575717703
NPV: 0.5071872571872571
